In [1]:
%matplotlib inline

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from chspy import join
from neurolib.models.multimodel import MultiModel
from neurolib.utils.stimulus import StepInput, ZeroInput
from scipy.optimize import curve_fit
from scipy.signal import butter, filtfilt, find_peaks, peak_widths
from scipy.stats import pearsonr

from hippocampus import HIPPOCAMPUS_NODE_DEFAULT_CONNECTIVITY, HippocampalCA3Node
from model_input import PeriodicRandomSquareInput
from utils import AnchoredHScaleBar

plt.style.use("default_light")

In [7]:
backend = "numba"
duration = 10
DT = 0.05
SAMPLING_DT = 0.1

hc = HippocampalCA3Node(aswr_mass_type="variable_depression", b_p_depression=True)
m = MultiModel.init_node(hc)
m.model_instance.initial_state = np.array([0.0001, 0.003, 0.012, 0.649])
m.params["backend"] = backend
m.params["duration"] = duration * 1e3
m.params["dt"] = DT
m.params["sampling_dt"] = SAMPLING_DT
m.run()